<a href="https://colab.research.google.com/github/LIMWAER/Bus-schedule-API/blob/master/Bus_schedule_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = "https://mybuses.ru/moscow/"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
div = soup.find("div", class_="list-group")
div

<div class="list-group"> <a href="/moscow/bus/1/" title="Расписание автобуса №1 - МГУ (к/ст) - Севастопольский просп. (к/ст, выс.)"><b>1</b> - МГУ (к/ст) - Севастопольский просп. (к/ст, выс.)</a> <a href="/moscow/bus/2/" title="Расписание автобуса №2 - 4-й мкр. Митина (к/ст) - Метро `Тушинская` (к/ст)"><b>2</b> - 4-й мкр. Митина (к/ст) - Метро "Тушинская" (к/ст)</a> <a href="/moscow/bus/3/" title="Расписание автобуса №3 - Камчатская ул. (к/ст) - Метро `Бульв. Рокоссовского`"><b>3</b> - Камчатская ул. (к/ст) - Метро "Бульв. Рокоссовского"</a> <a href="/moscow/bus/4/" title="Расписание автобуса №4 - Силикатный з-д (к/ст) - Метро `Краснопресненская` (выс.)"><b>4</b> - Силикатный з-д (к/ст) - Метро "Краснопресненская" (выс.)</a> <a href="/moscow/bus/5/" title="Расписание автобуса №5 - 9-й мкр. Солнцева (к/ст) - Платф. Переделкино (выс.)"><b>5</b> - 9-й мкр. Солнцева (к/ст) - Платф. Переделкино (выс.)</a> <a href="/moscow/bus/7/" title="Расписание автобуса №7 - Ст. Перово (к/ст) - Метро `Па

# Прототип функции парсинга таблиц:

In [19]:
def parse_schedule(a, bus_number):
  url = "https://mybuses.ru"+a["href"]
  page = requests.get(url)
  soup = BeautifulSoup(page.text, "html.parser")
  dfs = []
  dic = {}

  def get_rows(table):
    rows = []
    for tr in table.find_all('tr'):
      row = [td.string for td in tr.find_all('td')]
      if len(row) == 0:
        continue
      rows.append(row)
    return rows


  def get_cols(table):
    cols = []  
    for th in table.find_all('th'):
      try:
        cols.append(th.a.string)
      except AttributeError:
        cols.append(th.string)
    return cols

  for table in soup.find_all('table'):
    df = pd.DataFrame(data = get_rows(table), columns=get_cols(table))
    dfs.append(df)

  dic[bus_number] = dfs
  #print(dfs) 
  return dic


In [20]:
%%time
for a in div.find_all('a'):
  bus_number = a.b.string
  res = parse_schedule(a, bus_number)
  break

CPU times: user 319 ms, sys: 402 µs, total: 319 ms
Wall time: 1.38 s


In [28]:
res['1'][0]

,График / Остановка,МГУ (к/ст),МГУ (пос.),"Метро ""Ломоносовский проспект""",МГУ,Мичуринский просп.,Менделеевская ул.,Д/к МГУ,Ул. Лебедева,"Метро ""Университет""",Ул. Строителей,Просп. Вернадского,"Ул. Крупской, 9",Ул. Крупской,Ул. Марии Ульяновой,"Ленинский просп., 93","Ул. Кравченко, 2",Ул. Кравченко,Ул. Гарибальди,МФЦ Ломоносовского района,Ул. Архитектора Власова,"Агентство недвижимости ""ИНКОМ"" - Зал ""Оркест","Метро ""Новые Черемушки""",Школа-интернат в„– 17,"Профсоюзная ул., 66",Худ. объединение им. Вучетича,"Метро ""Калужская""","Метро ""Калужская"" (южн.)","Ул. Обручева, 23",44-й квартал,Школа,"Ул. Бутлерова, 9",Ул. Бутлерова,"Профсоюзная ул., 86",З-д спецсплавов,Ул. Введенского,"Ул. Обручева, 63","Севастопольский просп. (к/ст, выс.)"
0,выходные,05:14,05:14,05:15,05:16,05:17,05:19,05:20,05:23,05:26,05:27,05:30,05:31,05:32,05:39,05:38,05:35,05:37,05:40,05:42,05:44,05:45,05:47,05:48,05:49,05:50,05:51,06:02,05:53,05:56,05:57,05:58,05:59,06:00,06:04,06:05,06:06,06:08
1,будни,05:31,05:32,05:33,05:34,05:35,05:38,05:39,05:42,05:45,05:46,05:49,05:50,05:51,05:58,05:57,05:54,05:56,05:59,06:01,06:03,06:04,06:06,06:07,06:08,06:09,06:10,06:21,06:12,06:15,06:16,06:17,06:18,06:19,06:23,06:24,06:25,06:27
2,выходные,05:47,05:47,05:48,05:49,05:50,05:52,05:53,05:56,05:59,06:00,06:03,06:04,06:05,06:12,06:11,06:08,06:10,06:13,06:15,06:17,06:18,06:20,06:21,06:22,06:23,06:24,06:35,06:26,06:29,06:30,06:31,06:32,06:33,06:37,06:38,06:39,06:41
3,будни,05:51,05:52,05:53,05:54,05:55,05:58,05:59,06:02,06:05,06:06,06:09,06:10,06:11,06:18,06:17,06:14,06:16,06:19,06:21,06:23,06:24,06:26,06:27,06:28,06:29,06:30,06:41,06:32,06:35,06:36,06:37,06:38,06:39,06:43,06:44,06:45,06:47
4,будни,06:11,06:12,06:13,06:14,06:15,06:18,06:19,06:22,06:25,06:26,06:29,06:30,06:31,06:38,06:37,06:34,06:36,06:39,06:41,06:43,06:44,06:46,06:47,06:48,06:49,06:50,07:01,06:52,06:55,06:56,06:57,06:58,06:59,07:04,07:05,07:07,07:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,будни,22:52,22:53,22:54,22:55,22:56,22:59,23:00,23:03,23:06,23:08,23:10,23:11,23:13,23:20,23:19,23:16,23:18,23:21,23:23,23:25,23:26,23:28,23:29,23:30,23:31,23:32,23:43,23:34,23:37,23:38,23:39,23:40,23:41,23:45,23:46,23:47,23:49
72,выходные,23:39,23:39,23:40,23:41,23:42,23:44,23:45,23:48,23:51,23:52,23:55,23:56,23:57,00:04,00:03,00:00,00:02,00:05,00:07,00:09,00:10,00:12,00:13,00:14,00:15,00:16,00:27,00:18,00:21,00:22,00:23,00:24,00:25,00:29,00:30,00:31,00:33
73,будни,23:44,23:45,23:46,23:47,23:48,23:51,23:52,23:55,23:58,00:00,00:02,00:03,00:05,00:12,00:11,00:08,00:10,00:13,00:15,00:17,00:18,00:20,00:21,00:22,00:23,00:24,00:35,00:26,00:29,00:30,00:31,00:32,00:33,00:37,00:38,00:39,00:41
74,выходные,00:26,00:26,00:27,00:28,00:29,00:31,00:32,00:35,00:38,00:39,00:42,00:43,00:44,00:51,00:50,00:47,00:49,00:52,00:54,00:56,00:57,00:59,01:00,01:01,01:02,01:03,01:14,01:05,01:08,01:09,01:10,01:11,01:12,01:16,01:17,01:18,01:20


С помощью функции [.to_json](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html) получим необходимый файл формата .json. Пример файла указан в репозитории. Нужно преобразовать в нужный тип данных все строки.

In [33]:
res['1'][0].to_json(orient = 'index', force_ascii=False)

'{"0":{"График \\/ Остановка":"выходные","МГУ (к\\/ст)":"05:14","МГУ (пос.)":"05:14","Метро \\"Ломоносовский проспект\\"":"05:15","МГУ":"05:16","Мичуринский просп.":"05:17","Менделеевская ул.":"05:19","Д\\/к МГУ":"05:20","Ул. Лебедева":"05:23","Метро \\"Университет\\"":"05:26","Ул. Строителей":"05:27","Просп. Вернадского":"05:30","Ул. Крупской, 9":"05:31","Ул. Крупской":"05:32","Ул. Марии Ульяновой":"05:39","Ленинский просп., 93":"05:38","Ул. Кравченко, 2":"05:35","Ул. Кравченко":"05:37","Ул. Гарибальди":"05:40","МФЦ Ломоносовского района":"05:42","Ул. Архитектора Власова":"05:44","Агентство недвижимости \\"ИНКОМ\\" - Зал \\"Оркест":"05:45","Метро \\"Новые Черемушки\\"":"05:47","Школа-интернат в„– 17":"05:48","Профсоюзная ул., 66":"05:49","Худ. объединение им. Вучетича":"05:50","Метро \\"Калужская\\"":"05:51","Метро \\"Калужская\\" (южн.)":"06:02","Ул. Обручева, 23":"05:53","44-й квартал":"05:56","Школа":"05:57","Ул. Бутлерова, 9":"05:58","Ул. Бутлерова":"05:59","Профсоюзная ул., 86":"